<a href="https://colab.research.google.com/github/yndtky/portforio/blob/main/%E3%82%A2%E3%82%A4%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%A0%E3%81%AE%E5%A3%B2%E4%B8%8A%E4%BA%88%E6%B8%AC%E3%81%A8%E6%96%BD%E7%AD%96%E6%8F%90%E6%A1%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***時系列モデルを活用した売上予測***

#**概要**
--------------------------------------------------------------------------------
kerasを使って３１アイスクリームの売上高の時系列予測を行ないます。

予測結果からマーケティング施策の提案を行ってみたいと思います。

#**第一部：分析編**
--------------------------------------------------------------------------------



**使用データと分析環境**

・ここでは31アイスの売上の時系列予測を行なうにあたり、どのようなデータを準備したのかについてまとめます。

・（）の中はデータの取得タイミングと取得元です。

・今回はkerasを勉強するところから始まったことなどもあり、最新の売上予測になっていません。
*   **目的変数**：31アイスの売上データ（四半期ごと、[31アイスHP](https://www.31ice.co.jp/contents/company/ir/index.html)）
*   **説明変数**：
  *   31Club*の会員者数（四半期ごと、[31アイスHP](https://www.31ice.co.jp/contents/company/ir/index.html)）
  *   東京都の消費支出データ（月ごと、[都民のくらしむき（月報）（年報）](https://www.toukei.metro.tokyo.lg.jp/seikei/sb-index.htm) 内の品目別時系列データ）
  *   東京都の最高気温データ（月ごと、[気象庁HP](https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_s3.php?prec_no=44&block_no=47662&year=&month=&day=&view=a2)）
*   データの使用期間：2014年～2023年
*   環境や分析ツールなど：使用言語Python、深層学習フレームワークkerasを使った時系列予測

※31Clubとは31アイスの会員制アプリのことであり、各種クーポンなど様々なサービスが受けられます。


ここで、今回予測する売上データについて少し見てみます。これは2014年～2023年の四半期ごと単体の売上高をグラフにしたものです。

![３１アイスクリームの売上(2008-2023)](https://github.com/yndtky/portforio/blob/main/31_quaterly_sales_full.png?raw=true)

直近２年で売上が増加しています。また、グラフでは少しわかりずらいですが、売上が１年で一番落ち込む直前が第４四半期です（Q4）。第３四半期では売上は最大になりますが、第２四半期よりは売れ行きは落ちているように見えます。（Q1→Q2の傾きよりもQ2→Q3の傾きが緩やか）

次に、東京都の公開している統計データ（都民のくらしむき（年報）の品目別時系列データ）からアイスクリーム・シャーベットの行を抜き出し、結合してアイスクリーム・シャーベットの消費支出データを作成します。

In [ ]:
#データの前処理
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import japanize_matplotlib
import pandas as pd
from matplotlib.ticker import FuncFormatter

folder_path = "/成果物作成/消費データ"

# フォルダ内のすべてのExcelファイルを検索
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# 空のリストを用意して、必要なデータを格納
extracted_data = []

# すべてのExcelファイルを読み込み、条件に合致するデータを抽出してリストに追加
for file in excel_files:
    df = pd.read_excel(file)

    # 左から5番目の列を使用して条件をチェック
    if df.shape[1] > 4:  # 列数が5以上か確認
        condition_rows = df[df.iloc[:, 4] == 'アイスクリーム・シャーベット']

        # 条件に合致する行の左から9列目から20列目の値を抽出し、データフレームとしてリストに追加
        for _, row in condition_rows.iterrows():
            extracted_data.append(row.iloc[8:20].to_frame())
    else:
        print(f"The file {file} does not have enough columns.")

# 抽出したデータフレームを列方向に結合
if extracted_data:
    combined_df = pd.concat(extracted_data, axis=1, ignore_index=True)
else:
    combined_df = pd.DataFrame()
# 最終行を除いたデータフレームを作成
if not combined_df.empty:
    combined_df = combined_df.iloc[:-1, :]
else:
    combined_df = pd.DataFrame()
combined_df.iloc[0,18]=499

# データフレームの設定例（仮データを生成）
years = [2002 + i for i in range(22)]  # 2002年から2023年まで
months = list(range(1, 13))
index = [(y, m) for y in years for m in months]
data = [1000 * i for i in range(len(index))]  # 仮データ生成
final_df.index = pd.MultiIndex.from_tuples(index, names=['Year', 'Month'])

# 数値をカンマ区切りの形式で表示する関数
def thousand_comma(x, pos):
    return f'{int(x):,}'

# プロットの設定
plt.figure(figsize=(15, 5))
x_labels = [f'{year}_{month}' for year, month in index]
x_ticks = range(11, len(final_df), 12)  # 毎年の1月の位置に目盛りを設定
y_values = final_df.values

# プロット
plt.plot(range(len(final_df)), y_values)
plt.title('東京都のアイスクリームの消費')
plt.xlabel('年月')
plt.ylabel('消費額 (円)')
plt.xticks(x_ticks, [x_labels[i] for i in x_ticks], rotation=45)  # 年ごとの1月を横軸に設定
plt.xlim(left=0, right=len(final_df)-1)  # x軸の表示範囲をデータ範囲全体に設定
plt.gca().yaxis.set_major_formatter(FuncFormatter(thousand_comma))  # 縦軸のフォーマット設定
plt.grid(True)

plt.show()


東京都のアイスクリームの消費支出のデータは月ごとのデータであるため、横軸はyyyy/mm表記になっています。こちらも売上データ同様、直近２年の上昇トレンドが観察されます。

![東京都のアイスクリームの消費支出データ（勤労世帯＋無職世帯）](https://github.com/yndtky/portforio/blob/main/ice_consumption_tokyo_1.png?raw=true)


次に今回ポイントになる31club（会員制アプリ）の会員者数とその増加率の推移です。

会員数については一見順調に右肩上がりをしていますが、増加率を見てみると2019年Q4で大きく低下、2020年に持ち直すも、その後は下落トレンドに見えます。


![31アプリの会員者数](https://github.com/yndtky/portforio/blob/main/apli_members_4.png?raw=true)

※2021Q1については公表がなかったので、2020Q4と2021Q2の平均値として補間しました。ただ、IR資料に数値が載ってこないということは、公表できないくらいの値だったのか、それとも単に社内でアプリ会員者数の数値が重要視されていなかったのか（ローンチ初期で、そこまで良い数字でないならわざわざ公表することもないだろう）など、色々な理由を考えることはできます。

今回は主に次の理由から、前後の会員者数の平均値で値を代用することにしました。
*   マーケディング施策の内容を時系列モデルに組み込んでいない
*   増加率に上下はあるものの、常にプラスを保っていること



#***Kerasによる売上予測（LSTMモデルを使用）***
--------------------------------------------------------------------------------
以上の準備のもと、３１アイスクリームの2023年の売上を予測します。



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
np.random.seed(42)
tf.random.set_seed(42)

# 四半期ごとの売上データの読み込み
quarterly_sales = pd.read_csv("/成果物作成/202406～/売上データ_データ拡張後.csv", encoding='utf-8').values

# 四半期ごとの売上データのスケーリング
scaler_quarterly = MinMaxScaler()
quarterly_sales_scaled = scaler_quarterly.fit_transform(quarterly_sales.reshape(-1, 1))
quarterly_sales_scaled_df = pd.DataFrame(data=quarterly_sales_scaled, columns=['四半期売上'])

# 月次の気温データの正規化
scaler = MinMaxScaler()
monthly_temperature_scaled = scaler.fit_transform(df['最高気温(℃)の平均'].to_numpy().reshape(-1, 1))
monthly_temperature_scaled_df = pd.DataFrame(data=monthly_temperature_scaled, columns=['最高気温(℃)の平均'])

# 月次の消費データの正規化
scaler = MinMaxScaler()
monthly_consumption_scaled = scaler.fit_transform(df['アイスクリーム・シャーベットの消費'].to_numpy().reshape(-1, 1))
monthly_consumption_scaled_df = pd.DataFrame(data=monthly_consumption_scaled, columns=['アイスクリーム・シャーベットの消費'])

# アプリ会員数データの正規化
# 文字列型に変換
apli_members['アプリ会員数'] = apli_members['アプリ会員数'].astype(str)
# 'アプリ会員数'列が文字列型の場合のみ、カンマの除去と空白のトリムを行い、数値型に変換
if df['アプリ会員数'].dtype == 'object':
    df['アプリ会員数'] = df['アプリ会員数'].str.replace(',', '').str.strip().astype(float)
scaler = MinMaxScaler()
apli_members_scaled = scaler.fit_transform(df['アプリ会員数'].to_numpy().reshape(-1, 1))
apli_members_scaled_df = pd.DataFrame(data=apli_members_scaled, columns=['アプリ会員数'])


# インデックスをリセット
monthly_consumption_scaled_df.reset_index(drop=True, inplace=True)
monthly_temperature_scaled_df.reset_index(drop=True, inplace=True)
apli_members_scaled_df.reset_index(drop=True, inplace=True)
# thirty_one_flag.reset_index(drop=True, inplace=True)

df = pd.concat([monthly_consumption_scaled_df, monthly_temperature_scaled_df,apli_members_scaled_df], axis=1)

# LSTMモデルに入力するための形状変換
time_steps = 3  # 3か月分を1つのデータポイントとする
X_lstm = []
y_sales = []

monthly_data = df

# 四半期の数を計算
num_quarters = len(quarterly_sales_scaled)

# 最後の3か月を無視してインデックスエラーを回避
for i in range(0, len(monthly_data) - time_steps + 1, time_steps):
    if (i // time_steps) < num_quarters:  # 四半期の数を超えないようにする
        X_lstm.append(df.iloc[i:i + time_steps].values)
        # 対応する四半期の売上データを取得
        quarter_index = i // time_steps  # 3か月ごとに売上データが1つずつあるため、インデックスを計算
        y_sales.append(quarterly_sales_scaled[quarter_index])
    else:
        break  # 範囲外に出たらループを終了

X_lstm = np.array(X_lstm)
y_sales = np.array(y_sales)

# 時系列データを訓練データとテストデータに分割する
split_index = 56 #データが2009_1～2023_4(60個)あるので、2023_4までの56個のデータを訓練に使う

X_train, X_test = X_lstm[:split_index], X_lstm[split_index:]
y_train, y_test = y_sales[:split_index], y_sales[split_index:]


# LSTMモデルの構築
model = Sequential()
model.add(LSTM(32,activation='relu', input_shape=(time_steps, 3), return_sequences=True))
model.add(Dropout(0.1))  # 10%のドロップアウト
model.add(LSTM(32,activation='relu', input_shape=(time_steps, 3), return_sequences=True))
model.add(Dropout(0.1))  # 10%のドロップアウト

model.add(LSTM(32,))
model.add(Dropout(0.1))
model.add(Dense(1))

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# モデルの訓練
history = model.fit(X_train, y_train, epochs=300, batch_size=4, validation_data=(X_test, y_test), callbacks=[early_stopping]) #パッチサイズを(１年分)とした

# 学習の履歴を取得
history = history.history

plt.plot(history['loss'],label='loss')
plt.plot(history['val_loss'],label='val_loss')
plt.title('学習結果（アプリ会員数込）')
plt.xlabel('epoch')  # X軸のラベル
plt.ylabel('mean_squared_error')  # Y軸のラベル
plt.legend()
# PNGとして保存
plt.savefig('LSTM_training.png')
plt.show()


![LSTMモデルによる学習](https://github.com/yndtky/portforio/blob/main/LSTM_training_2.png?raw=true)

終始ギザギザしていますが、概ね200epochくらいから安定してきています。

In [ ]:
quarters = ['2023_1', '2023_2', '2023_3', '2023_4']
plt.plot(quarters,y_pred_inverse,label='predicted_sales')
plt.plot(quarters,y_test_inverse,label='actual_sales')
plt.title('売上予測（アプリ会員数込）')
plt.legend()
plt.show()


![LSTMモデルによる予測](https://github.com/yndtky/portforio/blob/main/LSTM_prediction_4.png?raw=true)

2023年の売上予測になります。

2023Q4 (2023/10～2023/12) の予測がやや上振れする結果になりました。

--------------------------------------------------------------------------------

**ここから下はアプリなし版の予測モデルについてです**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
np.random.seed(42)
tf.random.set_seed(42)

# 四半期ごとの売上データの読み込み
quarterly_sales = pd.read_csv("/成果物作成/202406～/売上データ_データ拡張後.csv", encoding='utf-8').values

# 四半期ごとの売上データのスケーリング
scaler_quarterly = MinMaxScaler()
quarterly_sales_scaled = scaler_quarterly.fit_transform(quarterly_sales.reshape(-1, 1))
quarterly_sales_scaled_df = pd.DataFrame(data=quarterly_sales_scaled, columns=['四半期売上'])


# 月次の気温データの正規化
scaler = MinMaxScaler()
monthly_temperature_scaled = scaler.fit_transform(df['最高気温(℃)の平均'].to_numpy().reshape(-1, 1))
monthly_temperature_scaled_df = pd.DataFrame(data=monthly_temperature_scaled, columns=['最高気温(℃)の平均'])

# 月次の消費データの正規化
scaler = MinMaxScaler()
monthly_consumption_scaled = scaler.fit_transform(df['アイスクリーム・シャーベットの消費'].to_numpy().reshape(-1, 1))
monthly_consumption_scaled_df = pd.DataFrame(data=monthly_consumption_scaled, columns=['アイスクリーム・シャーベットの消費'])

# インデックスをリセット
monthly_consumption_scaled_df.reset_index(drop=True, inplace=True)
monthly_temperature_scaled_df.reset_index(drop=True, inplace=True)
apli_members_scaled_df.reset_index(drop=True, inplace=True)

df = pd.concat([monthly_consumption_scaled_df, monthly_temperature_scaled_df]
               # ,apli_members_scaled_df
               , axis=1)

# LSTMモデルに入力するための形状変換
time_steps = 3  # 3か月分を1つのデータポイントとする
X_lstm_without_app = []
y_sales_without_app = []

monthly_data = df

# 四半期の数を計算
num_quarters = len(quarterly_sales_scaled)

# 最後の3か月を無視してインデックスエラーを回避
for i in range(0, len(monthly_data) - time_steps + 1, time_steps):
    if (i // time_steps) < num_quarters:  # 四半期の数を超えないようにする
        X_lstm_without_app.append(df.iloc[i:i + time_steps].values)
        # 対応する四半期の売上データを取得
        quarter_index = i // time_steps  # 3か月ごとに売上データが1つずつあるため、インデックスを計算
        y_sales_without_app.append(quarterly_sales_scaled[quarter_index])
    else:
        break  # 範囲外に出たらループを終了

X_lstm_without_app = np.array(X_lstm_without_app)
y_sales_without_app = np.array(y_sales_without_app)

# 時系列データを訓練データとテストデータに分割する
split_index = 56 #データが2009_1～2023_4(60個)あるので、2022_4までの56個のデータを訓練に使う

X_train_without_app, X_test_without_app = X_lstm_without_app[:split_index], X_lstm_without_app[split_index:]
y_train_without_app, y_test_without_app = y_sales_without_app[:split_index], y_sales_without_app[split_index:]


# LSTMモデルの構築
model_without_app = Sequential()
model_without_app.add(LSTM(32,activation='relu', input_shape=(time_steps, 2), return_sequences=True))
model_without_app.add(Dropout(0.1))  # 10%のドロップアウト
model_without_app.add(LSTM(32,activation='relu', input_shape=(time_steps, 2), return_sequences=True))
model_without_app.add(Dropout(0.1))  # 10%のドロップアウト

model_without_app.add(LSTM(32,))
model_without_app.add(Dropout(0.1))
model_without_app.add(Dense(1))

model_without_app.compile(optimizer=RMSprop(learning_rate=0.001), loss='mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# モデルの訓練
history_without_app = model_without_app.fit(X_train_without_app, y_train_without_app, epochs=300, batch_size=4, validation_data=(X_test_without_app, y_test_without_app), callbacks=[early_stopping]) #パッチサイズを(１年分)とした

# 学習の履歴を取得
history_without_app = history_without_app.history

plt.plot(history_without_app['loss'],label='loss_without_app')
plt.plot(history_without_app['val_loss'],label='val_loss_without_app')
plt.legend()
plt.show()


![LSTMモデルによる学習_アプリなし](https://github.com/yndtky/portforio/blob/main/LSTM_training_without_app_3.png?raw=true)

In [ ]:
# 予測と評価
y_pred_without_app = model_without_app.predict(X_test_without_app)
y_pred_inverse_without_app = scaler_quarterly.inverse_transform(y_pred_without_app)
y_test_inverse = scaler_quarterly.inverse_transform(y_test_without_app)

# 結果の表示
print("予測値:", y_pred_inverse_without_app)
print("実際の値:", y_test_inverse)


In [ ]:
quarters = ['2023_1', '2023_2', '2023_3', '2023_4']
plt.plot(quarters,y_pred_inverse_without_app,label='predicted_sales_without_app')
plt.plot(quarters,y_test_inverse,label='actual_sales')
plt.legend()
# PNGとして保存
plt.savefig('LSTM_prediction_without_app.png')
plt.show()


![LSTMモデルによる予測_アプリなし](https://github.com/yndtky/portforio/blob/main/LSTM_prediction_without_app_3.png?raw=true)

In [ ]:
import matplotlib.ticker as ticker
import japanize_matplotlib
quarters = ['2023Q1', '2023Q2', '2023Q3', '2023Q4']
plt.plot(quarters,y_test_inverse,label='actual_sales')
plt.plot(quarters,y_pred_inverse,label='predicted_sales')
plt.plot(quarters,y_pred_inverse_without_app,label='predicted_sales_without_app')
plt.title('アイスクリームの売上予測（単位：10億円）')
plt.legend()
# PNGとして保存
plt.savefig('LSTM_prediction_merge.png')
plt.show()


![LSTMモデルによる予測_結合](https://github.com/yndtky/portforio/blob/main/LSTM_prediction_merge_2.png?raw=true)

最終結果になります。売上実績（青線）に対し、アプリ込みモデルの予測（オレンジ）が良い近似になっています。一方で、アプリなしモデル（緑）では一年を通じて予測が下ぶれています。アプリ会員数の増加と売上増加は因果関係にあるかどうかはわかりませんが、会員数が売上予測に寄与していることは読み取れそうです。